In [1]:
%pip install --upgrade pip
%pip install sentence-transformers
%pip install transformers
%pip install datasets
%pip install faiss
%pip install torch
%pip install transformers[torch]
%pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pi

In [2]:
import pandas as pd
from datasets import load_dataset

import torch
from sentence_transformers import SentenceTransformer, models

from sklearn.manifold import MDS
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.image as mpimg
from transformers import BertTokenizer, BertForTokenClassification
import torch

from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import os
from datasets import load_metric

/home/vip/victor.henrique/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-29 16:49:50.154097: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 16:49:50.199225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 16:49:51.028540: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Carregando o corpus
dataset = load_dataset('mac_morpho')
dataset['train'] = dataset['train'].select(range(200)) # TODO: Remover
dataset['validation'] = dataset['validation'].select(range(200)) # TODO: Remover
dataset['test'] = dataset['test'].select(range(200)) # TODO: Remover

max_token_length = max([len(tokens) for tokens in dataset['train']['tokens']])

# Carregando o tokenizador
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

def encode_examples(examples):
    # Juntando os tokens em uma única string para cada exemplo
    # texts = examples['tokens']
    texts = [tokens for tokens in examples['tokens']]
    texts = ' '.join(texts)
    encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=max_token_length)
    
    labels = examples['pos_tags']
    if len(labels) < max_token_length:
        labels = labels + [-100]*(max_token_length-len(labels))

    elif len(labels) > max_token_length:
        labels = labels[:max_token_length]

    encodings['labels'] = labels

    # labels = examples['pos_tags']
    # encodings['labels'] = labels

    print(texts)
    print(type(texts[0]))
    print(encodings['input_ids'])
    print(labels)

    print(len(encodings['input_ids']))
    print(len(encodings['labels']))
    print("\n")
    return encodings



encoded_dataset = dataset.map(encode_examples)


In [21]:
# Sample text
text = "Casa"

# Encode text
output = tokenizer.encode_plus(text)

# Print output
print(output)
print(tokenizer.convert_ids_to_tokens(output['input_ids']))

# Decode input_ids
decoded_text = tokenizer.decode(output['input_ids'])

# Print decoded text
print(decoded_text)

{'input_ids': [101, 3052, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
['[CLS]', 'Casa', '[SEP]']
[CLS] Casa [SEP]


In [4]:
# print([len(tokens) for tokens in encoded_dataset['train']['tokens']])
# # print(min(len(encoded_dataset['train']['tokens'])))

# Decodificando as primeiras 5 sequências codificadas
for i in range(5):
    print(f"Sequência {i+1}:")
    size = encoded_dataset['train']['input_ids'][i]
    size2 = encoded_dataset['train']['token_type_ids'][i]
    print(size)
    print(size2)
    print("\n")


Sequência 1:
[101, 4166, 15335, 11903, 2517, 125, 12064, 109, 205, 117, 678, 6271, 229, 5304, 180, 13820, 5954, 173, 710, 1033, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Sequência 2:
[101, 1072, 10326, 22279, 327, 4036, 353, 14448, 1580, 171, 6539, 694, 117, 179, 2991, 644, 1906, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Sequência 3:
[101, 617, 4221, 4732, 122, 16232, 125, 10640, 7656, 22287, 5140,

In [27]:
print(encoded_dataset['train']['input_ids'][0])
print(encoded_dataset['train']['labels'][0])
print(len(dataset['train'].features['pos_tags'].feature.names))
# Definindo o modelo
class POSModel(BertForTokenClassification):
    def forward(self, input_ids, attention_mask, labels):
        outputs = super().forward(input_ids, attention_mask, labels)
        loss = outputs.loss
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        return loss, predictions

model = POSModel.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=len(dataset['train'].features['pos_tags'].feature.names))
# model = model.to('cpu')
# Definindo os argumentos de treinamento
args = TrainingArguments(
    "test-pos",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Definindo o treinador
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
)

# Treinando o modelo

print("Fine-tunning...")
trainer.train()

[101, 4166, 15335, 11903, 2517, 125, 12064, 109, 205, 117, 678, 6271, 229, 5304, 180, 13820, 5954, 173, 710, 1033, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[14, 19, 14, 15, 22, 7, 14, 9, 14, 9, 3, 15, 3, 3, 24, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
26


Some weights of POSModel were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
